# PCFG Simulator tutorial

7/14/19

Language and Cognitive Architecture Lab
Sean Anderson, Research Assistant

A quick tutorial for pcfg.py, a PCFG simulator that reads in a custom PCFG and generates sentences with corresponding Information metrics.

These metrics are supported:

* Surprisal (lexical)
* Lexical Entropy (over words)
* Structural Entropy (over meanings)
* Lexical Entropy Reduction
* Structural Entropy Reduction
* Kulback-Liebler Divergence, both (Pk+1 || Pk) and (Pk || Pk+1) (structural)
* Mutual Information (structural)

For formulas, justification, and discussion about some of these metrics, please see Hale (2001), Hale (2016), and Levy (2008).

Before starting, you need the following in your working directory:

```
pcfg.py
artificial_grammar.py
corpus.py
YOUR_CUSTOM_PCFG.txt
```

This program uses a Corpus of all legal sentences from the grammar provided. Generating this might take a while, but if you'd like to save a copy of the Corpus for future use you should have ```pickle``` installed. ```pickle``` is a Python module that writes and reads large Python objects to/from a file. Learn more [here](https://docs.python.org/3/library/pickle.html "Python3 Pickle docs").

## the custom grammar

Custom grammars are defined in a text file. 
Define your grammer as a bunch of rules, one per line, in the following format:

```P(rule) NonterminalSymbol -> Symbol1 Symbol2 Symbol3 ...```

Where 'NonterminalSymbol' is the parent symbol where this rule can be applied, and P(rule) is the probability that this rule is applied on the parent symbol.

There are a number of restrictions enforced on the grammar, including that only one recursive rule is allowed, and that the probabilites of rules using the same parent symbol must sum to 1.

Below is an example of a recursive PCFG:

The PCFG is implemented using artificial_grammar.py.

## quickly, how it works

The program reads in the custom grammar defined in the text file. Then it generates a Corpus of all possible legal sentences using the grammar, up to a certain recursion depth if specified. Using this Corpus, the program can calculate each of the complexity metrics at each word (i.e. timestep) in a sentence. 

Since generating all the sentences can be time and memory intensive, the Corpus is saved as a ```pickle``` file in the current directory. If ```pcfg.py``` is run with the the ```-p``` or ```--pickle``` flag, the program searches for an existing pickle file (.corpus). If there's one with the name as the grammar file, that corpus is used (skipping generating a new one).

## generating a sentence

To generate a single sentence, run with the --single/-s flag:

In [1]:
%run pcfg.py PCFG_flat.txt CP --single

Rules
=====
parent: CP
distr: [0.5, 0.5]
exp's: [('S', 'R', 'S'), ('Triangle', 'LeftOf', 'Circle')]

parent: S
distr: [0.5, 0.5]
exp's: [('Diamond',), ('Square',)]

parent: R
distr: [0.333, 0.333, 0.333]
exp's: [('Above',), ('Below',), ('RightOf',)]


metric    word  surprisal  entropy_lex  entropy_struct    ER_lex  ER_struct  \
i                                                                             
0       Square   2.000722     1.499750        2.791584  0.000000   0.206622   
1        Below   1.584963     1.584963        2.584963  0.584963   1.584963   
2       Square   1.000000     1.000000        1.000000  1.000000   1.000000   

metric  KL_diverge_PQ  KL_diverge_QP  
i                                     
0            2.000722            inf  
1            1.584963            inf  
2            1.000000            inf  


PCFG_generator/corpus.py:534: RuntimeWarning: divide by zero encountered in log2
  divergence += (2 ** logp) * (logp - np.log2(actualq))


_Note: every once in a while, numpy.log2 results in a RuntimeWarning when dividing by zero.  -np.inf is returned in these situtations._

_Another Note: because nplog2(0) = np.inf, KL_diverge_QP always evaulates to np.inf. This part needs to be fixed._

_Note 3: sentences are selected uniformly from corpus, not from their likelihood given the PCFG._

The PCFG_flat.txt was read, and the rules are printed. A .corpus pickle save was generated (```PCFG_flat.corpus```), as well as a csv storing the data from the sentence (```sentence.csv```):

In [3]:
%ls

PCFG Simulator Tutorial.ipynb  corpus.py
PCFG_example.txt               pcfg.py
PCFG_flat.corpus               sentence.csv
PCFG_flat.txt                  test.py
__pycache__/                   test.py~
artificial_grammar.py


The usage message for the program should also be informative.

In [4]:
%run pcfg.py --help

usage: pcfg.py [-h] [-l LIMIT_DEPTH] [-p | -g] [-s]
               grammar_file initial_symbol

Simulate sentence generation from a Custom PCFG,with Information Theoretic
Metrics

positional arguments:
  grammar_file          custom grammar.txt file
  initial_symbol        symbol used to start generation

optional arguments:
  -h, --help            show this help message and exit
  -l LIMIT_DEPTH, --limit_depth LIMIT_DEPTH
                        set depth limit for recursive generation
  -p, --pickle          use already existing Corpus pickle file
  -g, --generate        force generation of Corpus using grammar
  -s, --single          generate only one sentence


```sentence.csv``` stores the same data printed when running the program. It is important to note that the first row of the table represents the metrics _before_ reading the first word. For example, with a context of 0 words, I have a Surprisal of ~2 bits when I see the first word is _Square_ (given PCFG_flat). My Lexical Entropy before reading the word _Square_ is ~1.5 bits, and my Lexical Entropy Reduction from seeing no words to having read _Square_ is 0. This follows similarly for Structural Entropy and Entropy Reduction.

Importantly, this means that in the last row the Surprisal column displays the Surprisal upon reading the last word in the sentence, while the Lexical Entropy displays the entropy right before reading the last word.

## caveats

As the program is currently designed, if the grammar is altered but the name is kept, then generating the corpus will overwrite the previous corpus (unless ```--pickle``` is passed). If ```--pickle``` is passed, then the pickle .corpus file made with the old grammar will be used, not the new grammar. The program does not effectively keep track of which corpus comes from which grammar. Similarly, every time ```--single``` is passed, sentence.csv will be overwritten with the new sentence.

## coming soon

* Generation without ```--single``` flag: I'm still working on a way to have a csv effectively contain the stats for all the sentences of the corpus, with their likelihoods given the grammar. This would enable generation in one go. 
* Mutual Information calculation
* For ambiguity and meaning, see below.
* More efficient sentence generation. Right now stacks and sentence-in-makings are duplicated across stack frames, wasting lots of memory. I'm trying to figure out a way to not have that happen, and make generating the corpus faster.

## how pcfg.py handles ambiguous grammars

Every time a sentence is generated using the grammar, its likelihood is stored. As ambiguous grammars can generate the same sentence in different ways, we might end up with 2 or more likelihoods for a single sentence. In the corpus, each of these likelihoods are stored as a 'SyntaxTree' in a list linked to the sentence. Any sentence that has more than one SyntaxTree (effectively, likelihood) linked to it is treated as ambiguous. Structural Entropy and Lexical Entropy are then calculated accordingly, the former by summing over each possible meaning (i.e. SyntaxTree) and the latter by each sentence (all likelihoods pooled into one).

Right now, SyntaxTree only stores the likelihood of one possible generation of the sentence, but the program could easily be modified to store more things about the meaning of the sentence.